# Depth-vs-Performance Study
This notebook runs Nelder-Mead optimizations for `p>1` on both MaxCut and 2-SAT instances, mirroring the trends highlighted in Willsch et al.

In [ ]:
from pathlib import Path
import sys

REPO_ROOT = Path.cwd().resolve().parent
SRC_ROOT = REPO_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))

: 

In [ ]:
import numpy as np
import pandas as pd
from problems.maxcut import generate_maxcut_instance
from problems.sat import generate_2sat_instance
from qaoa.circuits import CostHamiltonian, maxcut_hamiltonian
from qaoa.experiments import run_depth_experiment
from visualization.landscapes import plot_optimization_progress

In [ ]:
maxcut_instance = generate_maxcut_instance(6, 0.5, weighted=False, seed=11)
maxcut_h = maxcut_hamiltonian(maxcut_instance.weights, maxcut_instance.n_nodes)
sat_instance = generate_2sat_instance(6, 10, ensure_satisfiable=True, seed=21)
sat_h = CostHamiltonian(sat_instance.h, sat_instance.J, sat_instance.n_variables)
print(f"MaxCut ground energy: {maxcut_instance.ground_state_energy:.3f}")
print(f"2-SAT ground energy: {sat_instance.ground_energy:.3f}")

In [ ]:
p_values = [1, 2, 3]
records = []
histories = {}
for label, hamiltonian, ground_states, e_min, e_max in [
    ("MaxCut", maxcut_h, maxcut_instance.ground_state_bitstrings, maxcut_instance.ground_state_energy, maxcut_instance.energy_ceiling),
    ("2-SAT", sat_h, sat_instance.ground_states, sat_instance.ground_energy, 0.0),
]:
    for p in p_values:
        result = run_depth_experiment(
            hamiltonian=hamiltonian,
            ground_states=ground_states,
            energy_min=e_min,
            energy_max=e_max,
            p=p,
            n_restarts=5,
            shots=256,
        )
        best_run = min(result.optimization_runs, key=lambda r: r.value)
        histories[(label, p)] = best_run.history
        records.append(
            {
                "Problem": label,
                "p": p,
                "Mean energy": result.analysis.energy_mean,
                "Success": result.analysis.success_probability,
                "Approximation": result.analysis.approximation_ratio,
            }
        )
results_df = pd.DataFrame(records)
results_df

In [ ]:
pivot = results_df.pivot(index="p", columns="Problem", values="Success")
ax = pivot.plot(marker="o", figsize=(6, 4))
ax.set_ylabel("Success probability")
ax.set_xlabel("Circuit depth p")
ax.set_title("Success vs depth")
ax.grid(True)

In [ ]:
for (label, p), history in histories.items():
    if p == max(p_values):
        print(f"{label} optimization history (p={p})")
        plot_optimization_progress(
            history,
            ground_energy=maxcut_instance.ground_state_energy if label == "MaxCut" else sat_instance.ground_energy,
        )

## Takeaways
- Success probabilities climb with depth but remain problem dependent (2-SAT lags MaxCut).
- Optimal regions remain narrow, requiring multiple restarts.
- Convergence traces highlight plateaus consistent with the paper's findings.